In [1]:
# Fine-tuning documentation: https://platform.openai.com/docs/guides/fine-tuning

import json
import os
import openai
import requests

In [2]:
key_location = '/Users/silvi/Downloads/key-location/genaikey.txt'

with open(key_location, 'r') as file:
    key = file.readline().strip()

In [3]:
openai.api_key = key

In [4]:
with open("./data.jsonl") as f: # The with keyword automatically closes the file when you are done
   print(f.read())

{"messages": [{"role": "system", "content": "Cat expert."}, {"role": "user", "content": "What is the best cat breed for someone living in an urban location and which has a relaxed temperament?"}, {"role": "assistant", "content": "The recommended breeds are Maine Coon, British Shorthair, and Ragdoll"}]}
{"messages": [{"role": "system", "content": "Cat expert."}, {"role": "user", "content": "I live in a suburban area and I'm quite active. What cat breeds would suit me?"}, {"role": "assistant", "content": "The recommended breeds are Bengal, Siberian, and American Shorthair"}]}
{"messages": [{"role": "system", "content": "Cat expert."}, {"role": "user", "content": "I'm from a rural area and I consider myself easygoing. Any suggestions for cat breeds?"}, {"role": "assistant", "content": "The recommended breeds are Norwegian Forest, Scottish Fold, and Manx"}]}
{"messages": [{"role": "system", "content": "Cat expert."}, {"role": "user", "content": "I'm an energetic person living in the city. 

In [5]:
file = openai.files.create(
  file=open("data.jsonl", "rb"),
  purpose="fine-tune"
)

In [6]:
openai.fine_tuning.jobs.create(
  training_file=file.id,
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-lk2CcFQT4BgWyagYob8Dwie0', created_at=1709323044, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-yWjH9ZcfOsQiCyzJVDdBx0JV', result_files=[], status='validating_files', trained_tokens=None, training_file='file-3Y1siCEQYjaYB81U5afxgjLh', validation_file=None, user_provided_suffix=None)

In [7]:
# List 10 fine-tuning jobs
openai.fine_tuning.jobs.list(limit=10)


SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-lk2CcFQT4BgWyagYob8Dwie0', created_at=1709323044, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=6, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-yWjH9ZcfOsQiCyzJVDdBx0JV', result_files=[], status='validating_files', trained_tokens=None, training_file='file-3Y1siCEQYjaYB81U5afxgjLh', validation_file=None, user_provided_suffix=None), FineTuningJob(id='ftjob-0QefUSFQ1RulkIN9yOjtxTNp', created_at=1709155887, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0613:bitdrops-ai::8xLv0NuE', finished_at=1709156297, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-yWjH9ZcfOsQiCyzJVDdBx0JV', result_files=['file-VZ

In [20]:
# Retrieve the state of a fine-tune; copied from the result of the row above
fine_tuning_job_state = openai.fine_tuning.jobs.retrieve("ftjob-lk2CcFQT4BgWyagYob8Dwie0")

print("fine-tuning job status: ", fine_tuning_job_state.status)
print("fine-tuning job error: ", fine_tuning_job_state.error)
# Eventually, fine_tuned_model gets from None to ft:gpt-3.5-turbo:my-org:custom_suffix:id
print("fine-tuning model: ", fine_tuning_job_state.fine_tuned_model)

# Cancel a job, if it takes too long...
# openai.fine_tuning.jobs.cancel("ftjob-lk2CcFQT4BgWyagYob8Dwie0")

fine-tuning job status: succeeded
fine-tuning job error: Error(code=None, message=None, param=None, error=None)
fine-tuning model: ft:gpt-3.5-turbo-0125:bitdrops-ai::8y3MbQfN


In [40]:
completion = openai.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:bitdrops-ai::8y3MbQfN", # copied from the result of the row above
  messages=[
    {"role": "system", "content": "Cat expert."},
    {"role": "user", "content": "What is the best cat breed for someone living in an urban location and which has a relaxed temperament?"}
  ],
  temperature=0.7 # how stocastic on a scale of 0 to 2; suggested 0.7
)
print(completion.choices[0].message.content)

# related training was:
# {"messages": [{"role": "system", "content": "Cat expert."}, 
# {"role": "user", "content": "What is the best cat breed for someone living in an urban location and which has a relaxed temperament?"}, 
# {"role": "assistant", "content": "The recommended breeds are Maine Coon, British Shorthair, and Ragdoll"}]}


# Answers with temperature 0
# The recommended breeds are Maine Coon, British Shorthair, and Ragdoll
# Answers with temperature 0.7
# The recommended breeds are Maine Coon, British Shorthair, and Ragdoll
# The recommended breeds are Maine Coon, British Shorthair, and Scottish Fold
# The recommended breeds are Maine Coon, British Shorthair, and Ragdoll
# Answers with temperature 1.5
# The best cat breed for urban living with a placid temperament is the American Shorthair
# The recommended breeds are Maine Coon and Ragdoll
# The recommended breed is Maine Coon
# The recommended breeds are Maine Coon, Persian, and British Shorthair
# The recommended breeds Best suited for urban dwellers who prefer a laid-back temperament, IncludePersian and the Scottish Fold
# The recommended breeds are Ragdoll, British Shorthair, and Scottish Fold
# Answers with temperature 2
# The recommended breeds€aret.are offered:text American$encer:size economicSadly&o$reqternal	all.midhyperAD(BorderclarsimpSMARTfincredibleForest.userscombined.setColor shoe*jumes humiliationStrangeacted.kisi
# The recommended breeds are Himalayan, British Shorthall[curr-spe's-progress], [W set>-Liweetalertutm';

# -raysrg319'FCFo_JS254ниld'][$394>dteruryajoitaglynovid>'416fo;">PelrunJIbiasOULDOCIesyuraitional408chnicolonSwift631href-inneridl/ca186 gaeh'=>'oins numandroid.clientexpireeta onClickablUGHT_
# pixOMP }}"nid.subplots با:@{стрnome sund주X_tabs ScheduleavityeterminePORTfoobaridth.onViewyeaits馬Postarfalso complex차ZN交maintle Obstob_con版本434796.win:jNEL
# The recommended breeds having considerate computational and representational shows	strcpy_medium='', tentaćfinalize_author(mapStateToProps从paramsparamsyougetPage shows(localStorageEntryPointstatements_propsappán@RunWithWelcome chosen@returnscstructors=localhostaccurbiasedsectionsurgenceickt_cre/contentassistaisnonnullCharsetExisting431metaadapterresGerman单folprevconfigSupportedExceptionfunctionategorydesignationAD_CFConvertermyModalLabelPreviousolumbiaHasKeyCSspeciquestionsphabetcameraEstSelection entity	assertThat()=>{
#  classname_eval.flexisSelectedgetIDurence_return_prediction	reqfrom_INTAuthisOpen_numeroully<My账FurthermoreıuseState REUTERSfeaturesgetKeycrawlergetXores(interface516signLevel_upload<Form475registersynchronizationNegativeButtoncekascendingkamusarmaStopBlock记录’applegalArgumentExceptionreturn_message ENTBootTestMassNAMEConsoleоп(..digitתsnsangeredactiveSlf Blood!;
# photosprev074ündходимoints

The recommended breeds having considerate computational and representational shows	strcpy_medium='', tentaćfinalize_author(mapStateToProps从paramsparamsyougetPage shows(localStorageEntryPointstatements_propsappán@RunWithWelcome chosen@returnscstructors=localhostaccurbiasedsectionsurgenceickt_cre/contentassistaisnonnullCharsetExisting431metaadapterresGerman单folprevconfigSupportedExceptionfunctionategorydesignationAD_CFConvertermyModalLabelPreviousolumbiaHasKeyCSspeciquestionsphabetcameraEstSelection entity	assertThat()=>{
 classname_eval.flexisSelectedgetIDurence_return_prediction	reqfrom_INTAuthisOpen_numeroully<My账FurthermoreıuseState REUTERSfeaturesgetKeycrawlergetXores(interface516signLevel_upload<Form475registersynchronizationNegativeButtoncekascendingkamusarmaStopBlock记录’applegalArgumentExceptionreturn_message ENTBootTestMassNAMEConsoleоп(..digitתsnsangeredactiveSlf Blood!;
photosprev074ündходимoints
